In [ ]:
import pandas as pd


In [ ]:
!ls

In [ ]:
!ls ../data/results

In [ ]:
#df_cpu_1024 = pd.read_csv('../data/results/cpu_1024_full_temp00_math_stats.csv')
#df_gpu_1024_8b_nomath = pd.read_csv('../data/results/gpu_1024_8b_temp00_nomath_stats.csv')
#df_gpu_1024_4b_math = pd.read_csv('../data/results/gpu_1024_4b_temp00_math.csv')
#df_gpu_1024_4b_nomath = pd.read_csv('../data/results/gpu_1024_4b_temp00_nomath.csv')

df_cpu_1024 = pd.read_csv('../data/results/cpu_1024_full_temp00_math_stats.csv')
df_gpu_1024_8b = pd.read_csv('../data/results/gpu_1024_8b_v2.csv')
df_gpu_1024_4b = pd.read_csv('../data/results/gpu_1024_4b_v3.csv')


In [ ]:
df_cpu_1024.head()

In [ ]:
# extract python code from the answer column in df_cpu_1024
df_cpu_1024['python_code'] = df_cpu_1024['answer'].str.extract(r'```python\n(.*)```', expand=False)
df_gpu_1024_8b['python_code'] = df_gpu_1024_8b['answer'].str.extract(r'```python\n(.*)```', expand=False)
df_gpu_1024_4b['python_code'] = df_gpu_1024_4b['answer'].str.extract(r'```python\n(.*)```', expand=False)
#df_gpu_1024_4b_nomath['python_code'] = df_gpu_1024_4b_nomath['answer'].str.extract(r'```python\n(.*)```', expand=False)

In [ ]:
import re

def slice_until_word(s, word):
    start_index = s.find(word)
    if start_index == -1:
        # word not found in string
        return s
    end_index = start_index + len(word)
    return s[end_index:]


def slice_function_definition(completion, words):
    completion = slice_until_word(completion, "<|assistant|>")

    lines = completion.split('\n')  # split the string into lines
    result = []
    capture = False

    for line in lines:
        if line:
            if len(line.strip()) > 0:
                first_word = line.split()[0]
                if first_word in words:
                    capture = True
                elif capture and (line[0].isalpha() or line[0] == '`'):
                    capture = False
        if capture:
#            print(line)
            result.append(line)

    return '\n'.join(result)

def extract_python_code(input_string):
#    if "```python" in input_string:
#        python_code = re.findall("```python(.*?)```", input_string, re.DOTALL)
#        answers = [code.strip() for code in python_code]
#        return answers[0]
    return slice_function_definition(input_string, ["from", "import", "def"])


In [ ]:
print(extract_python_code((df_cpu_1024['answer'][0])))

In [ ]:
def create_python_code_column(df):
    python_codes = df['answer'].apply(extract_python_code)
    df['python_code'] = python_codes
    #return df

In [ ]:
create_python_code_column(df_cpu_1024)
create_python_code_column(df_gpu_1024_8b)
create_python_code_column(df_gpu_1024_4b)
#create_python_code_column(df_gpu_1024_4b_nomath)

In [ ]:
df_gpu_1024_8b.head()

In [ ]:
df_python_codes = pd.concat([df_cpu_1024['python_code'],
                             df_gpu_1024_8b['python_code'],
                             df_gpu_1024_4b['python_code']],
                             #df_gpu_1024_4b_nomath['python_code']],
                            axis=1)

In [ ]:
df_python_codes.columns = ['cpu_1024', 'gpu_1024_8b', 'gpu_1024_4b']
df_python_codes.head()


In [ ]:
df_passes = pd.concat([df_cpu_1024['pass'], df_gpu_1024_8b['pass'], df_gpu_1024_4b['pass']], axis=1)
df_passes.columns = ['cpu_1024', 'gpu_1024_8b', 'gpu_1024_4b']

In [ ]:
df_passes.head()

In [ ]:
df_passes.columns

In [ ]:
# plot df_passes value counts as bar plot
df_passes["cpu_1024"].value_counts().plot(kind='bar', color=['red', 'green'])


In [ ]:
# plot df_cpu_1024 columns pass values counts as bar plot
df_cpu_1024["pass"].value_counts().plot(kind='bar', color=['red', 'green'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming df_passes is your DataFrame
cols = ['cpu_1024', 'gpu_1024_8b', 'gpu_1024_4b']

# Calculate the value counts for 0 and 1 for each column
data = []
for col in cols:
    data.append(df_passes[col].value_counts().sort_index().tolist())

data = np.array(data).T

# Create bar plot
N = len(cols)
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots()

p1 = ax.bar(ind - width/2, data[0], width, color='r')
p2 = ax.bar(ind + width/2, data[1], width, color='b')

ax.axhline(0, color='grey', linewidth=0.8)
ax.set_ylabel('Counts')
ax.set_title('Counts by group and variable')
ax.set_xticks(ind)
ax.set_xticklabels(cols)

# add counts above bars
for rect in p1 + p2:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2., 1.002*height,
            '%d' % int(height), ha='center', va='bottom')

# turn x-ticks 90 degrees
plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='center')

#ax.legend((p1[0], p2[0]), ('fail', 'pass'))

# Move the legend to the bottom using bbox_to_anchor
ax.legend((p1[0], p2[0]), ('fail', 'pass'), bbox_to_anchor=(1, 0.9), loc='upper center', ncol=1)

plt.show()


In [ ]:
print(df_cpu_1024["python_code"][0])


In [ ]:
print(df_gpu_1024_8b["python_code"][0])

In [ ]:
print(df_gpu_1024_4b["python_code"][0])

In [ ]:
print(df_gpu_1024_4b["python_code"][0])

In [ ]:
!pwd

In [ ]:
%cd ../data/starchat-beta

In [ ]:
!ls

In [ ]:
%cd human-eval

In [ ]:
!pip install -e .

In [ ]:
from human_eval.data import write_jsonl, read_problems
from human_eval import execution

problems = read_problems()

In [ ]:
df_cpu_1024.columns

In [ ]:
%cd /mystuff/data

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("starchat-beta", local_files_only=True)


In [ ]:
from tqdm.notebook import tqdm

def collect_results(df_from, tokenizer):
    task_ids = []
    execution_passed = []
    execution_passed_with_import_math = []
    execution_passed_with_import_typing = []
    execution_results = []
    execution_results_with_import_math = []
    execution_results_with_import_typing = []
    prompt_token_counts = []
    generated_code_token_counts = []
    answer_token_counts = []
    answer_new_token_counts = []
    token_diff_counts = []
    canonicals = []
    prompts = []
    for idx, row in tqdm(df_from.iterrows()):
        problem = problems[row.problem_id]
        canonicals.append(problem["canonical_solution"])
        prompt = problem["prompt"]
        prompts.append(prompt)
        prompt_tokens = tokenizer.tokenize(prompt)
        prompt_token_count = len(prompt_tokens)
        prompt_token_counts.append(prompt_token_count)
        generated_code = row.python_code
        generated_code_tokens = tokenizer.tokenize(row.python_code)
        generated_code_token_count = len(generated_code_tokens)
        generated_code_token_counts.append(generated_code_token_count)
        answer_tokens = tokenizer.tokenize(row.answer)
        answer_token_count = len(answer_tokens)
        generated_code_new_token_counts = answer_token_count - prompt_token_count
        answer_new_token_counts.append(generated_code_new_token_counts)
        answer_token_counts.append(answer_token_count)
        answer_new_token_count = answer_token_count - prompt_token_count - generated_code_token_count
        answer_new_token_counts.append(answer_new_token_count)
        token_diff_counts.append(answer_token_count - prompt_token_count)
        #10 is timeout in seconds, 1 is optional completion_id
        result = execution.check_correctness(problem, generated_code, 10, 1)
        execution_passed.append(result["passed"])
        execution_results.append(result["result"])
        generated_code = "import math\n"+generated_code
        result = execution.check_correctness(problem, generated_code, 10, 1)
        execution_passed_with_import_math.append(result["passed"])
        execution_results_with_import_math.append(result["result"])
        generated_code = "from typing import List, Optional\n"+generated_code
        result = execution.check_correctness(problem, generated_code, 10, 1)
        execution_passed_with_import_typing.append(result["passed"])
        execution_results_with_import_typing.append(result["result"])
        task_ids.append(result["task_id"])
    df_from['canonical'] = canonicals
    df_from['prompt'] = prompts
    df_new = pd.DataFrame({'problem_id': task_ids, 'passed_no_math': execution_passed, 'passed_math': execution_passed_with_import_math,
                           'passed_typing': execution_passed_with_import_typing,
                           'result_no_math': execution_results, 'result_math': execution_results_with_import_math,
                           'result_typing': execution_results_with_import_typing,
                           'prompt_token_count': prompt_token_counts, 'generated_code_token_count': generated_code_token_counts,
                           'answer_token_count': answer_token_counts})
    merged_df = df_from.merge(df_new, on='problem_id')
    return merged_df


In [ ]:
df_gpu_1024_8b["pass"].value_counts()

In [ ]:
df_cpu_1024 = collect_results(df_cpu_1024, tokenizer)
df_gpu_1024_8b = collect_results(df_gpu_1024_8b, tokenizer)
df_gpu_1024_4b = collect_results(df_gpu_1024_4b, tokenizer)

In [ ]:
df_cpu_1024["passed_math"].value_counts()

In [ ]:
df_gpu_1024_4b["passed_math"].value_counts()

In [ ]:
df_gpu_1024_8b["passed_math"].value_counts()


In [ ]:
df_cpu_1024.head()

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!mkdir processed

In [ ]:
%cd processed

In [ ]:
df_cpu_1024.to_csv('cpu_1024_v2.csv')

In [ ]:
df_gpu_1024_4b.to_csv('gpu_1024_4b_v2.csv')

In [ ]:
df_gpu_1024_8b.to_csv('gpu_1024_8b_v2.csv')

In [ ]:
%cd /mystuff/data/processed

In [ ]:
!ls

In [ ]:
df_cpu_1024 = pd.read_csv('cpu_1024_v2.csv')

In [ ]:
df_gpu_1024_4b = pd.read_csv('gpu_1024_4b_v2.csv')


In [ ]:
df_gpu_1024_8b = pd.read_csv('gpu_1024_8b_v2.csv')

In [ ]:
df_cpu_1024.head()

In [ ]:
df_cpu_1024.plot.scatter(x='prompt_token_count', y='answer_token_count')

In [ ]:
df_cpu_1024.plot.scatter(x='prompt_token_count', y='generated_code_token_count')

In [ ]:
df_cpu_1024.plot.scatter(x='generated_code_token_count', y='answer_token_count')

In [ ]:
# find row with highest value for answer_token_count in df_cpu_1024
max_ans = df_cpu_1024['answer_token_count'].idxmax()
df_cpu_1024.loc[max_ans]


In [ ]:
print(df_cpu_1024.loc[max_ans]["result_math"])

In [ ]:
max_ans = df_cpu_1024['generated_code_token_count'].idxmax()
df_cpu_1024.loc[max_ans]


In [ ]:
print(df_cpu_1024.loc[max_ans]["python_code"])

In [ ]:
df_cpu_1024.loc[max_ans].problem_id.replace("/", "_")

In [ ]:
print(df_cpu_1024.loc[max_ans]["test"])

In [ ]:
%cd /mystuff/data
!mkdir codes_cpu
!mkdir codes_gpu4b
!mkdir codes_gpu8b

In [ ]:
from tqdm.notebook import tqdm

def save_codes(df_from, folder_name):
    for idx, row in tqdm(df_from.iterrows()):
        print(row.problem_id)
        print(row.python_code)
        print(row.test)
        entry_point = problems[row.problem_id]["entry_point"]
        print(entry_point)

        with open(folder_name+"/"+row.problem_id.replace("/", "_")+".py", "w") as f:
            f.write(row.python_code)
            f.write("\n\n##########################################################\n\n")
            f.write(row.test)
            f.write("\n\n##########################################################\n\n")
            f.write(f"check({entry_point})")

        with open(folder_name+"/answer_"+row.problem_id.replace("/", "_")+".txt", "w") as f:
            f.write(row.answer)


In [ ]:
save_codes(df_cpu_1024, "codes_cpu")
save_codes(df_gpu_1024_4b, "codes_gpu4b")
save_codes(df_gpu_1024_8b, "codes_gpu8b")


In [ ]:
!ls codes_cpu

In [ ]:
!ls codes_gpu4b

In [ ]:
!ls codes_gpu8b

In [ ]:
!cat HumanEval_0.py

In [ ]:
problems["HumanEval/0"]["entry_point"]

In [ ]:
df_cpu_1024.head()

In [ ]:
df_cpu_1024["passed_math"].value_counts()

In [ ]:
df_cpu_1024["passed_typing"].value_counts()

In [ ]:
print(df_cpu_1024["python_code"].iloc[12])

In [ ]:
df_cpu_1024["passed_no_math"].iloc[12]

In [ ]:
#find row with highest length of answer


In [ ]:
df_cpu_1024["answer_token_count"].idxmax()

In [ ]:
#get top 10 rows with highest length of answer
df_cpu_1024.nlargest(10, 'answer_token_count').index
